# Two Tower Model Implementation in PyTorch

## Content-based filtering with a Two Tower neural network

This notebooks aims to implement the a content-based filtering algorithm for a movie recommendation system using the two tower approach. Based on the implementation of Andrew NG [Link to Tensorflow Implementation](https://github.com/john2408/Machine-Learning-Specialization-Coursera/blob/main/C3%20-%20Unsupervised%20Learning%2C%20Recommenders%2C%20Reinforcement%20Learning/week2/C3W2/C3W2A2/C3_W2_RecSysNN_Assignment.ipynb). However, we aim to leverage the Pytorch landscape, and implement the example using the pytorch lighting library. 

In [6]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tabulate
import sys, os

sys.path.insert(0, os.path.abspath('..'))

from src.movie_dataset_utils import load_data
pd.set_option("display.precision", 1)

### Content-based filtering with a Two Tower neural network

<figure>
    <center> <img src="./images/RecSysNN.png"   style="width:500px;height:280px;" ></center>
</figure>

The Two-Tower model is a neural network architecture used for recommendation systems. It consists of two separate neural networks (towers) that learn user and item representations independently. The user tower processes user features, while the item tower processes item features. The outputs of these towers are then combined, typically using a similarity measure like cosine similarity, to predict the relevance score or rating for a given user-item pair. This model allows for efficient retrieval of recommendations by precomputing item representations.


## Movie ratings dataset 
The data set is derived from the [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/) dataset. 

[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

The original dataset has 9000 movies rated by 600 users with ratings on a scale of 0.5 to 5 in 0.5 step increments. The dataset has been reduced in size to focus on movies from the years since 2000 and popular genres. The reduced dataset has $n_u = 395$ users and $n_m= 694$ movies. For each movie, the dataset provides a movie title, release date, and one or more genres. For example "Toy Story 3" was released in 2010 and has several genres: "Adventure|Animation|Children|Comedy|Fantasy|IMAX".  This dataset contains little information about users other than their ratings. This dataset is used to create training vectors for the neural networks described below. 

## Data Analysis

In [7]:
# Load Data, set configuration variables
cwd = os.getcwd()
path = os.path.join(cwd, "data/gold/movie_dataset/")
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data(path=path)


num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
scaledata = True  # applies the standard scalar to data if true
print(f"Number of training vectors: {len(item_train)}")

FileNotFoundError: /Users/JOHTORR/Repos/two_tower_model/notebooks/data/gold/movie_dataset/content_item_train.csv not found.

In [3]:
len(user_train), len(item_train), len(y_train)

(58187, 58187, 58187)

In [4]:
len(item_vecs)

1883

In [5]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9


In [6]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8798,2004,3.8,0,0,0,0,0,1,0,0,0,0,0,0,0,0


## Two Tower Model

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import f1_score
import numpy as np
from torch.optim import Adam
from einops import rearrange


# class CrossMultiHeadAttention(MultiHeadAttention):
#     def __init__(self, *args, **kwargs):
#         """
#         See MultiHeadAttention.
#         Takes a cross hidden as query. Needs to be same dimension!
#         """
#         super().__init__(*args, **kwargs)

#         self.to_qkv = torch.nn.Linear(self.dim_input, self.dim_head * self.nr_heads * 2, bias=False)
#         self.to_q = torch.nn.Linear(self.dim_input, self.dim_head * self.nr_heads, bias=False)

#     def forward(self, x: torch.Tensor, cross_hidden: torch.Tensor, attn_mask: torch.Tensor = None) -> torch.Tensor:
#         assert cross_hidden.shape[-1] == x.shape[-1], "Input and cross input require same embedding dimension!"

#         h = self.nr_heads
#         k, v = self.to_qkv(x).chunk(2, dim=-1)
#         q = self.to_q(cross_hidden)
#         q, k, v = map(lambda t: rearrange(t, "b n (h d) -> b h n d", h=h), (q, k, v))
#         sim = torch.einsum("b h i d, b h j d -> b h i j", q, k) * self.scale

#         if attn_mask is not None:
#             attn_mask = attn_mask.unsqueeze(1)  # Add an extra dimension for the heads (b, 1, i, j)
#             sim = sim.masked_fill(attn_mask == 0, float("-inf"))

#         attn = sim.softmax(dim=-1)
#         attn = self.dropout(attn)

#         out = torch.einsum("b h i j, b h j d -> b h i d", attn, v)
#         out = rearrange(out, "b h n d -> b n (h d)", h=h)
        
#         return self.to_out(out)


In [291]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(
        self,
        dim_input: int,
        nr_heads: int = 8,
        dropout_p: float = 0.0,
        scale_factor: float = 0.5,
    ):
        """Ye olde Multihead Attention, implmented with Einstein Notation.
        Note: There' ain't no masking here, so be careful!

        Args:
            dim_input (int): The input dimension
            nr_heads (int, optional): Number of heads. Defaults to 8.
            dropout_p (float, optional): Dropout. Defaults to 0.0.
            scale_factor (float, optional): Exponent of the scaling division - default is square root. Defaults to 0.5.
        """
        super().__init__()
        self.nr_heads = nr_heads
        self.dim_input = dim_input
        self.dim_head = dim_input // nr_heads
        self.scale = self.dim_head**-scale_factor

        self.to_qkv = torch.nn.Linear(dim_input, self.dim_head * self.nr_heads * 3, bias=False)

        self.to_out = torch.nn.Linear(self.dim_head * nr_heads, dim_input)
        self.dropout = torch.nn.Dropout(dropout_p)

    def forward(self, x: torch.Tensor, attn_mask: torch.Tensor = None) -> torch.Tensor:
        h = self.nr_heads
        q, k, v = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, "b n (h d) -> b h n d", h=h), (q, k, v))
        sim = torch.einsum("b h i d, b h j d -> b h i j", q, k) * self.scale

        if attn_mask is not None:
            attn_mask = attn_mask.unsqueeze(1)  # Add an extra dimension for the heads (b, 1, i, j)
            sim = sim.masked_fill(attn_mask == 0, float("-inf"))

        attn = sim.softmax(dim=-1)
        attn = self.dropout(attn)

        out = torch.einsum("b h i j, b h j d -> b h i d", attn, v)
        out = rearrange(out, "b h n d -> b n (h d)", h=h)
        return self.to_out(out)


class UserTower(nn.Module):
    def __init__(self, input_dim, embed_dim, output_dim, nr_heads):
        super(UserTower, self).__init__()
        self.embed_dim = embed_dim
        # Input Embedding Layer (projects 4D input to embed_dim)
        self.embedding = nn.Linear(input_dim, embed_dim)
        # Multi-Head Attention Layer
        self.attention = MultiHeadAttention(dim_input=embed_dim, nr_heads=nr_heads)
        # Fully Connected Layer (maps back to scalar output)
        self.fc = nn.Linear(embed_dim, output_dim)

    def forward(self, x, attn_mask=None):
        x = self.embedding(x)  # Project to higher dimension
        x = x.unsqueeze(1)  # Add sequence dimension: (batch, seq_len=1, embed_dim)
        attn_output = self.attention(x, x)   
        x = attn_output.mean(dim=1)  # Average over sequence dimension
        return self.fc(x) 

class ProductTower(nn.Module):
    def __init__(self, input_dim, embed_dim, output_dim, nr_heads):
        super(ProductTower, self).__init__()
        self.embed_dim = embed_dim
        # Input Embedding Layer (projects 4D input to embed_dim)
        self.embedding = nn.Linear(input_dim, embed_dim)
        # Multi-Head Attention Layer
        self.attention = MultiHeadAttention(dim_input=embed_dim, nr_heads=nr_heads)
        # Fully Connected Layer (maps back to scalar output)
        self.fc = nn.Linear(embed_dim, output_dim)

    def forward(self, x, attn_mask=None):
        x = self.embedding(x)  # Project to higher dimension
        x = x.unsqueeze(1)  # Add sequence dimension: (batch, seq_len=1, embed_dim)
        attn_output = self.attention(x, x)   
        x = attn_output.mean(dim=1)  # Average over sequence dimension
        return self.fc(x) 


class UserTower(nn.Module):
    def __init__(self, input_dim, embed_dim, output_dim, nr_heads, window_size=3):
        super(UserTower, self).__init__()
        self.window_size = window_size  # Window size for rolling
        # Input Embedding Layer (projects 4D input to embed_dim)
        self.embedding = nn.Linear(input_dim, embed_dim)
        # Multi-Head Attention Layer
        self.attention = MultiHeadAttention(dim_input=embed_dim, nr_heads=nr_heads)
        # Fully Connected Layer (maps back to scalar output)
        self.fc = nn.Linear(embed_dim, output_dim)

    def rolling_window(self, x, window_size):
        """
        Creates overlapping sequences using a rolling window approach.
        """
        batch_size, embed_dim = x.shape
        if embed_dim < window_size:
            raise ValueError("Embed_dim should be greater than or equal to window_size")

        return x.unfold(dimension=1, size=window_size, step=1)  # (batch_size, embed_dim-window_size+1, window_size)

    def forward(self, x):
        x = torch.relu(self.embedding(x))  # Apply first linear layer with activation
        x = self.rolling_window(x, self.window_size)  # Create overlapping sequences
        # Multihead Attention expects (batch_size, seq_len, hidden_dim) -> (seq_len, batch_size, hidden_dim)
        x = x.permute(0, 2, 1)  
        attn_output = self.attention(x, x)   
        x = attn_output.permute(1, 0, 2)  # Convert back to (batch, seq_len, hidden_dim)
        x = self.output_layer(x.mean(dim=1))  # Pooling + Final Linear Layer
        return x
    
class UserTower_MLP(nn.Module):
    def __init__(self, input_dim, embed_dim, output_dim, nr_heads):
        super(UserTower_MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, embed_dim)
        self.fc2 = nn.Linear(embed_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

class ProductTower_MLP(nn.Module):
    def __init__(self, input_dim, embed_dim, output_dim):
        super(ProductTower_MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, embed_dim)
        self.fc2 = nn.Linear(embed_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

class TwoTowerRecommendationModel(pl.LightningModule):
    def __init__(self, user_config, product_config, learning_rate):
        super(TwoTowerRecommendationModel, self).__init__()
        self.user_tower = UserTower(**user_config)
        self.product_tower = ProductTower(**product_config)
        self.criterion = nn.MSELoss()
        self.learning_rate = learning_rate

    def forward(self, user_input, product_input):
        
        product_output = self.product_tower(product_input)
        user_output = self.user_tower(user_input)
        rating_pred_tensor = F.cosine_similarity(user_output, product_output)
        return rating_pred_tensor

    def training_step(self, batch, batch_idx):
        user_input, product_input, target = batch
        rating_pred_tensor = self(user_input, product_input)
        loss = self.criterion(rating_pred_tensor, target)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        user_input, product_input, target = batch
        rating_pred_tensor = self(user_input, product_input)
        loss = self.criterion(rating_pred_tensor, target)
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        user_input, product_input, target = batch
        rating_pred_tensor = self(user_input, product_input)
        loss = self.criterion(rating_pred_tensor, target)
        self.log('test_loss', loss)
        return {'preds': rating_pred_tensor, 'target': target}

    def predict_step(self, batch, batch_idx):
        user_input, product_input, _ = batch
        rating_pred_tensor = self(user_input, product_input)
        return rating_pred_tensor

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=self.learning_rate)

# Generate Random Sample Data
def generate_random_sample_data(num_samples: int):
    """This function generates random sample data for the two-tower model.
    The idea is to understand how to structure the data for the model.

    Args:
        num_samples (int): number of samples

    Returns:
        _type_: _description_
    """
    user_input_dim = 6
    product_input_dim = 3
    user_data = np.random.rand(num_samples, user_input_dim)
    product_data = np.random.rand(num_samples, product_input_dim)
    target_data = np.random.randint(0, 2, size=(num_samples, 1))  # Binary targets for F1 score

    user_tensor = torch.tensor(user_data, dtype=torch.float32)
    product_tensor = torch.tensor(product_data, dtype=torch.float32)
    target_tensor = torch.tensor(target_data, dtype=torch.float32)

    return user_tensor, product_tensor, target_tensor

In [292]:
use_synthetic_data = False
if use_synthetic_data:
    # Generate synthetic data
    num_samples = 1000
    user_data, product_data, target_data = generate_random_sample_data(num_samples)

In [293]:
# Analyze the shape of the data from movie lens dataset
#user_data.shape, product_data.shape, target_data.shape

In [294]:
# scale training data
scaledata = True
if scaledata:
    item_train_save = item_train
    user_train_save = user_train
    y_train_save = y_train

    scalerItem = StandardScaler()
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)

    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)
    
    targetScaler = MinMaxScaler((-1, 1))
    targetScaler.fit(y_train.reshape(-1, 1))
    y_train = targetScaler.transform(y_train.reshape(-1, 1))

    print(np.allclose(item_train_save, scalerItem.inverse_transform(item_train)))
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_train)))

user_data_tensor = torch.tensor(user_train[:, u_s:], dtype=torch.float32)
product_data_tensor = torch.tensor(item_train[:, i_s:], dtype=torch.float32)
target_data_tensor = torch.tensor(y_train.reshape(-1,1), dtype=torch.float32) 

True
True


In [295]:
target_data_tensor.max(), target_data_tensor.min()

(tensor(1.), tensor(-1.))

In [296]:
user_data_tensor.shape, product_data_tensor.shape, target_data_tensor.shape

(torch.Size([58187, 14]), torch.Size([58187, 16]), torch.Size([58187, 1]))

In [297]:
user_data_tensor.shape

torch.Size([58187, 14])

In [298]:
# Hyperparameters
batch_size = 32
epochs = 5
learning_rate = 0.001
use_gpu = True

# Model configurations
user_config = {'input_dim': user_data_tensor.shape[1], 
               'embed_dim': 128,
               'output_dim': 64, 
               'nr_heads': 8}
product_config = {'input_dim': product_data_tensor.shape[1], 
                  'embed_dim': 128, 
                  'output_dim': 64,
                  'nr_heads': 8}

# Create DataLoader
dataset = TensorDataset(user_data_tensor, product_data_tensor, target_data_tensor)
num_samples = target_data_tensor.shape[0]
train_size = int(0.8 * num_samples)
test_size = num_samples - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize model
model = TwoTowerRecommendationModel(user_config, product_config, learning_rate)

if not use_gpu:

    # Train the model
    trainer = pl.Trainer(max_epochs=epochs)
    trainer.fit(model, train_loader, test_loader)

else:
    # Move model to GPU if available
    device = torch.device('mps' if torch.cuda.is_available() else 'mps')
    model.to(device)

    # Train the model on GPU
    trainer = pl.Trainer(max_epochs=epochs, accelerator="gpu", devices=1)
    trainer.fit(model, train_loader, test_loader)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type         | Params | Mode 
-------------------------------------------------------
0 | user_tower    | UserTower    | 75.8 K | train
1 | product_tower | ProductTower | 76.1 K | train
2 | criterion     | MSELoss      | 0      | train
-------------------------------------------------------
151 K     Trainable params
0         Non-trainable params
151 K     Total params
0.608     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/JOHTORR/Repos/two_tower_model/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


RuntimeError: linear(): input and weight.T shapes cannot be multiplied (3x126 and 128x384)

## Predictions for a new user
First, we'll create a new user and have the model suggest movies for that user. After you have tried this example on the example user content, feel free to change the user content to match your own preferences and see what the model suggests. Note that ratings are between 0.5 and 5.0, inclusive, in half-step increments.

In [275]:
new_user_id = 5000
new_rating_ave = 1.0
new_action = 1.0
new_adventure = 1
new_animation = 1
new_childrens = 1
new_comedy = 5
new_crime = 1
new_documentary = 1
new_drama = 1
new_fantasy = 1
new_horror = 1
new_mystery = 1
new_romance = 5
new_scifi = 5
new_thriller = 1
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

user_vecs = gen_user_vecs(user_vec,len(item_vecs))

In [276]:
user_vecs.shape, item_vecs.shape

((1883, 17), (1883, 17))

In [277]:
if scaledata:
    scaled_user_vecs = scalerUser.transform(user_vecs)
    scaled_item_vecs = scalerItem.transform(item_vecs)
    user_data_tensor = torch.tensor(scaled_user_vecs[:, u_s:], dtype=torch.float32)
    product_data_tensor = torch.tensor(scaled_item_vecs[:, i_s:], dtype=torch.float32)
    y_p = model(user_data_tensor, product_data_tensor).detach().numpy()
    y_p = targetScaler.inverse_transform(y_p.reshape(-1, 1))
else:
    y_p = model(user_vecs[:, u_s:], item_vecs[:, i_s:])

In [278]:
print("Prediction Vector Shape", y_p.shape)

Prediction Vector Shape (1883, 1)


In [279]:
if np.any(y_p < 0) : 
    print("Error, expected all positive predictions")
sorted_index = np.argsort(-y_p,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_p[sorted_index]
sorted_items = item_vecs[sorted_index]
sorted_user  = user_vecs[sorted_index]

In [280]:
y_p, user, item, movie_dict = sorted_ypu, sorted_user, sorted_items, movie_dict

maxcount=10
count = 0
movies_listed = defaultdict(int)
disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

for i in range(0, y_p.shape[0]):
    if count == maxcount:
        break
    count += 1
    movie_id = item[i, 0].astype(int)
    if movie_id in movies_listed:
        continue
    movies_listed[movie_id] = 1
    disp.append([y_p[i, 0], item[i, 0].astype(int), item[i, 2].astype(float),
                movie_dict[movie_id]['title'], movie_dict[movie_id]['genres']])

table = tabulate.tabulate(disp, tablefmt='html',headers="firstrow")

In [281]:
table

y_p,movie id,rating ave,title,genres
0.393598,43928,1.92308,Ultraviolet (2006),Action|Fantasy|Sci-Fi|Thriller
0.393595,31221,2.05,Elektra (2005),Action|Adventure|Crime|Drama
0.393595,6482,1.95455,Dumb and Dumberer: When Harry Met Lloyd (2003),Comedy
0.393595,34520,2.08333,"Dukes of Hazzard, The (2005)",Action|Adventure|Comedy
0.393595,4228,2.15,Heartbreakers (2001),Comedy|Crime|Romance
0.393595,46335,2.09091,"Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006)",Action|Crime|Drama|Thriller
0.393593,37380,2.15,Doom (2005),Action|Horror|Sci-Fi
